In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
import quadratic_weighted_kappa
from sklearn.metrics import make_scorer
from sklearn.cross_validation import train_test_split
import feature_generator
from scipy.stats import pearsonr

In [2]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    

In [3]:
def GetBestModel(modelsToUse):
    
    K = 3
    bestQwk = -1
    bestModel = None
    kf = KFold(len(dfTrain), K)
    meanTestQwk = 0
    meanTrainQwk = 0
    num = 1

    for train_index, test_index in kf:
        
        frames = [pd.read_csv('fold%s.csv' % str(i)) for i in range(1, K+1) if i != num  ]
        trainModelPredictions = pd.concat(frames)  
        
        xTrain = trainModelPredictions[modelsToUse].values
        yTrain = dfTrain.iloc[train_index]['Response']
                
        model = LinearRegression()
        model.fit(xTrain, yTrain)

        xTest = pd.read_csv('fold%s.csv' % str(num))[modelsToUse].values
        yTest = dfTrain.iloc[test_index]['Response']

        predictions = np.clip(np.rint(model.predict(xTest)).astype(int), 1, 8)
        trainPredictions = np.clip(np.rint(model.predict(xTrain)).astype(int), 1, 8)
        
        testQwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yTest)
        trainQwk = quadratic_weighted_kappa.quadratic_weighted_kappa(trainPredictions, yTrain)
        
        print "TestQWK: %s\n" % testQwk
        print "Train QWK: %s\n" % trainQwk
        
        meanTestQwk += (testQwk / K)
        meanTrainQwk += (trainQwk / K)
        
        if testQwk > bestQwk:
            bestQwk= testQwk
            bestModel = model  
            
        num += 1

    return bestModel, meanTestQwk, meanTrainQwk

In [26]:
def WritePredictionsToFile(stackingModel, modelsToUse, dfTest):
        X = pd.read_csv('testPredictions.csv')[modelsToUse].values
        predictions = np.clip(np.rint(stackingModel.predict(X)).astype(int), 1, 8)
        predDf = pd.DataFrame()
        predDf['Id'] = dfTest['Id']
        predDf['Response'] = predictions
        print predictions
        predDf.to_csv(path_or_buf='xgbCombo.csv', columns=['Id', 'Response'], index=False, header=['Id', 'Response'])

In [13]:
modelsToUse = ['LogisticRegression', 'BaggingLinearRegression_n_estimators=10', 'XGBoost', 'BaggingDescisionTrees_n_estimators=20']
#                ']
# modelsToUse = [
model, _, _ = GetBestModel(modelsToUse)

TestQWK: 0.605393839227

Train QWK: 0.608425411559

TestQWK: 0.605755770079

Train QWK: 0.609462186597

TestQWK: 0.611050074564

Train QWK: 0.604622406375



In [27]:
WritePredictionsToFile(model, modelsToUse, dfTest)

[3 6 6 ..., 4 3 3]


In [14]:
fold1 = pd.read_csv('fold1.csv')

for i in range(len(fold1.columns)):
    for j in range(i + 1, len(fold1.columns)):        
        print "%s, %s, %s" % (fold1.columns[i], fold1.columns[j], pearsonr(fold1[fold1.columns[i]], fold1[fold1.columns[j]]))

BaggingDescisionTrees_n_estimators=20, BaggingLinearRegression_n_estimators=10, (0.82499924867883212, 0.0)
BaggingDescisionTrees_n_estimators=20, LogisticRegression, (0.70820642260934052, 0.0)
BaggingDescisionTrees_n_estimators=20, XGBoost, (0.90503228130107249, 0.0)
BaggingLinearRegression_n_estimators=10, LogisticRegression, (0.8059315280605025, 0.0)
BaggingLinearRegression_n_estimators=10, XGBoost, (0.89666039594474478, 0.0)
LogisticRegression, XGBoost, (0.77818405579561756, 0.0)
